# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI(
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1'
)

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图描绘了一条穿越郁郁葱葱绿地的木板步道。图中的步道直直地延伸到远方，引导观者的目光向景深深处探去。四周被高高的绿色草地包围，远处可见一些树木和灌木丛。天空呈现出明亮的蓝色，散布着几朵白云，增添了一种宁静和平和的氛围。整体风景美丽宜人，透出一种平静与自然的和谐感。这种场景可能位于一个自然保护区或乡村公园，是进行散步和自然观察的理想场所。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图描绘了一条穿越郁郁葱葱绿地的木板步道。图中的步道直直地延伸到远方，引导观者的目光向景深深处探去。四周被高高的绿色草地包围，远处可见一些树木和灌木丛。天空呈现出明亮的蓝色，散布着几朵白云，增添了一种宁静和平和的氛围。整体风景美丽宜人，透出一种平静与自然的和谐感。这种场景可能位于一个自然保护区或乡村公园，是进行散步和自然观察的理想场所。'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(  
        # 代理地址，填写商家中转站或自建OpenAI代理
        base_url='https://api.xiaoai.plus/v1'
    )  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [4]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图包括两张搞笑或者说是恶搞性质的照片，每张里面有一只被极度漫画化的柴犬。左侧的柴犬被合成为一体格极其健壮的肌肉男身体，头部仍保留柴犬的特征。配文“16岁的我，工作后的我”，暗示青少年时期充满活力和力量，而成年后工作的压力使人变得沉重与疲惫。右侧的柴犬则表情显得疲软和无精打采，体现出工作后可能的生活状态。

这种图片一般用于表达幽默或社会讽刺，反映了工作压力对人们生活的影响，并用夸张的手法放大这种对比，以此来引发观者的共鸣或思考。


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [7]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(    
    # 代理地址，填写商家中转站或自建OpenAI代理
    base_url='https://api.xiaoai.plus/v1'
)  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post(f"{client.base_url}/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [8]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图显示了1980年至2020年间美国、中国、日本和德国的国内生产总值（GDP）的比较。坐标轴的横轴表示年份，纵轴表示GDP的数额，单位为万亿美元（Trillion USD）。

各国GDP走势如下：
- 蓝线代表美国，其GDP从1980年的较低水平逐年增长，到2020年达到最高。
- 红线代表中国，从1980年的非常低的起点开始，其增长速度在2000年后显著加快，到2020年接近美国的GDP。
- 紫线代表日本，1990年代初达到高峰后，GDP有所下降，此后幅度变化不大。
- 绿线代表德国，其GDP增长相对平稳，整体呈现逐年上升的趋势。

此图有效地展示了这四个经济体在过去40年间的经济增长和变化趋势。从图中可以看出中国的迅速崛起，以及其他国家的相对稳定或平缓的增长模式。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [14]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这是一张笔记本上的手写笔记。主题是关于使用在自然语言处理中，特别是在Transformer模型的参数调优方法。具体内容涉及到几种调优策略，包括Prompt Tuning、Prefix Tuning和LoRA。

1. **Prompt Tuning（提示调整）**：这种方法是为了适应小型模型，通过在模型输入的嵌入层添加固定的提示向量（一组特定的词嵌入，表示为X）来进行。

2. **Prefix Tuning（前缀调整）**：这中调整方法是通过向Transformer模型的输入前缀中添加可训练的参数W来改变模型的行为，简单地说就是修改模型输入部分的向量。

3. **LoRA（低秩适应）**：LoRA技术涉及修改现有模型参数，通过引入低秩矩阵（ΔW = AB，其中A和B分别是不同维度的矩阵）来适应模型，而不是完全训练所有参数，这种方法可以有效地减轻计算负担。

笔记还提到了关于这些技术的储存需求（例如，LoRA需要78GB，而QLoRA需要48GB），显示了不同技术在资源消耗上的差异。

这些笔记很可能是某个研究者或学生在学习或工作中对相关技术的理解与解析，用于帮助更好地掌握和应用这些自然语言处理技术。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [15]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这是一张笔记本上的手写笔记。主题是关于使用在自然语言处理中，特别是在Transformer模型的参数调优方法。具体内容涉及到几种调优策略，包括Prompt Tuning、Prefix Tuning和LoRA。

1. **Prompt Tuning（提示调整）**：这种方法是为了适应小型模型，通过在模型输入的嵌入层添加固定的提示向量（一组特定的词嵌入，表示为X）来进行。

2. **Prefix Tuning（前缀调整）**：这中调整方法是通过向Transformer模型的输入前缀中添加可训练的参数W来改变模型的行为，简单地说就是修改模型输入部分的向量。

3. **LoRA（低秩适应）**：LoRA技术涉及修改现有模型参数，通过引入低秩矩阵（ΔW = AB，其中A和B分别是不同维度的矩阵）来适应模型，而不是完全训练所有参数，这种方法可以有效地减轻计算负担。

笔记还提到了关于这些技术的储存需求（例如，LoRA需要78GB，而QLoRA需要48GB），显示了不同技术在资源消耗上的差异。

这些笔记很可能是某个研究者或学生在学习或工作中对相关技术的理解与解析，用于帮助更好地掌握和应用这些自然语言处理技术。

![](./images/handwriting_1.jpg)

In [18]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图展示的是一本笔记本上手写的学术笔记内容，主要涉及自然语言处理（NLP）和机器学习中的多种技术和方法。这些笔记看起来是关于如何改进和定制深度学习模型，尤其是Transformer模型的研究。

左手边的页码涉及以下几个主题：
1. **自然语言预训练** - 提到了自然语言预训练（PEFT、PFET、SOTA等术语）以及不同的预训练方法。
2. **Prompt Tuning** - 提及了使用adapter和各种Prompt Tuning技术（如Google和Stanford开发的技术）来微调模型。
3. **模型名称** - 列出了一些现代NLP模型名字如GPT、Bloom、ChatGLM、Alpaca、MOSS。

右手边的页码提到了：
1. **模型适配器和调节方法** - 对新适配器（如LoRA, QLoRA, AdLoRA, PETC）和prefix-tuning & Adapters的效能进行了探讨。
2. **多模态指令微调语言模型** - 提到了多模态语言模型，特别是LLaMA和LUMS。
3. **结构和插入形态学** - 讨论了如何在模型中添加结构以改进性能，包括如何处理attention和refine操作。

这些笔记可能是为了学术研究、会议准备或者相关技术博客编写的准备材料，展示了当前自然语言处理技术和方法的多样性和复杂性。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

##### 我的第一个作业
![](./images/homework_handwriting.jpg)

In [26]:
content = query_base64_image_description("./images/homework_handwriting.jpg", prompt='''
我将给你一些关键词（按照序列），你将识别图像中和关键词关联性较高的内容，并以JSON格式的文本返回，
JSON要求如下：1. JSON文本中的字段就是我给的关键词，2. 字段的值是一个对象，该对象有两个固定字段：value, analyze, 3. 将图像中关联性较高的内容自动填入对应的字段中value，将你的分析填入analyze，4. 无论value没有值，都必须给出analyze
我的关键词是：1. name, 2. birth_year, 3. origin_content
''')
print(content)


{
  "name": {
    "value": "Liam",
    "analyze": "The text explicitly mentions 'Liam is my name', indicating that the name associated with the subject in the image is Liam."
  },
  "birth_year": {
    "value": "",
    "analyze": "Although the image contains the text '2024 years', it does not specify a birth year, rather seems to mention a duration or a future date. It is unrelated to a birth year context."
  },
  "origin_content": {
    "value": "",
    "analyze": "The image does not provide any information regarding the origin content of any individual or subject."
  }
}


##### 我的第二个作业
![](./images/homework_handwriting.jpg)

In [27]:
markdown_content = query_base64_image_description("./images/homework_handwriting.jpg", prompt='''
我将给你一些关键词（按照序列），你将识别图像中和关键词关联性较高的内容，并以Markdown格式的文本返回，
Markdown要求如下：1. Markdown文本中的一级标题就是我给的关键词，2. 每个一级标题有两个二级标题：value, analyze, 3. 将图像中关联性较高的内容自动填入对应的value正文内容，将你的分析填入analyze正文内容，4. 无论value没有值，都必须给出analyze
我的关键词是：1. name, 2. birth_year, 3. origin_content

''')
display(Markdown(markdown_content))

# name
## value
Liam
## analyze
The image clearly shows handwritten text stating "Liam is my name," which directly provides the name as Liam.

# birth_year
## value
2024
## analyze
The text "2024 years" in the image appears to indicate a year, which could potentially be a birth year, but lacking additional context, it seems more a typographical error or a conceptual misunderstanding in the text.

# origin_content
## value

## analyze
The image does not provide any specific information or indications about the origin of the content in terms of geographical or cultural context, it only shows handwritten text related to a name and a numerical figure.